<a href="https://colab.research.google.com/github/9-coding/Google_ML_Bootcamp_5th/blob/main/NL2SQL_gemma2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Dependencies

In [2]:
!pip install --quiet datasets
!pip install --quiet huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your termin

In [4]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Load Dataset

In [5]:
dataset = load_dataset("Clinton/Text-to-sql-v1")
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/118 [00:00<?, ?B/s]

texttosqlv2.jsonl:   0%|          | 0.00/635M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/262208 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'response', 'source', 'text'],
        num_rows: 262208
    })
})

In [6]:
dataset['train'][0]

{'instruction': 'Name the home team for carlton away team',
 'input': 'CREATE TABLE table_name_77 (\n    home_team VARCHAR,\n    away_team VARCHAR\n)',
 'response': 'SELECT home_team FROM table_name_77 WHERE away_team = "carlton"',
 'source': 'sql_create_context',
 'text': 'Below are sql tables schemas paired with instruction that describes a task. Using valid SQLite, write a response that appropriately completes the request for the provided tables. ### Instruction: Name the home team for carlton away team ### Input: CREATE TABLE table_name_77 (\n    home_team VARCHAR,\n    away_team VARCHAR\n) ### Response: SELECT home_team FROM table_name_77 WHERE away_team = "carlton"'}

# Load Gemma 2

In [7]:
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2-2b-it")

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

## Prompt Format
```
<bos><start_of_turn>user
Write a hello world program<end_of_turn>
<start_of_turn>model
```

In [14]:
text = dataset['train']['text'][0]
input = dataset['train']['input'][0]
instruction = dataset['train']['instruction'][0]
stripped_text = text[:text.find("### Instruction:")].strip()
content = f"{stripped_text}\n{instruction}\n{input}"

print(content)

Below are sql tables schemas paired with instruction that describes a task. Using valid SQLite, write a response that appropriately completes the request for the provided tables.
Name the home team for carlton away team
CREATE TABLE table_name_77 (
    home_team VARCHAR,
    away_team VARCHAR
)


In [17]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)

messages = [
    {
        "role": "user",
        "content": content
    }
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [18]:
prompt

'<bos><start_of_turn>user\nBelow are sql tables schemas paired with instruction that describes a task. Using valid SQLite, write a response that appropriately completes the request for the provided tables.\nName the home team for carlton away team\nCREATE TABLE table_name_77 (\n    home_team VARCHAR,\n    away_team VARCHAR\n)<end_of_turn>\n<start_of_turn>model\n'

In [19]:
outputs = pipe(
    prompt,
    do_sample=True,
    temperature=0.2,
    top_k=50,
    top_p=0.95,
    add_special_tokens=True
)

In [21]:
print(outputs[0]["generated_text"][len(prompt):])

```sql
CREATE TABLE table_name_77 (
    home_team VARCHAR,
    away_team VARCHAR
);

-- Insert some sample data
INSERT INTO table_name_77 (home_team, away_team) VALUES
('Carlton', 'Melbourne');
```

**Explanation:**

1. **CREATE TABLE table_name_77:** This line creates a new table named `table_name_77`.
2. **home_team VARCHAR, away_team VARCHAR:** This defines two columns named `home_team` and `away_team`, both of type `VARCHAR` (for storing text).
3. **INSERT INTO table_name_77 (home_team, away_team) VALUES ('Carlton', 'Melbourne');** This line inserts a row into the table. It specifies the values for the `home_team` and `away_team` columns.

**Note:**

* Replace `table_name_77` with the actual name you want to use for your table.
* You can add more rows to the table by repeating this `INSERT` statement. 


Let me know if you have any other SQL tasks or questions. 



In [27]:
def generate_prompt(example):
  prompt_list = []
  for i in range(len(example['text'])):
    text = dataset['train']['text'][i]
    input = dataset['train']['input'][i]
    instruction = dataset['train']['instruction'][i]
    stripped_text = text[:text.find("### Instruction:")].strip()
    content = f"{stripped_text}\n{instruction}\n{input}"

    prompt_list.append(r"""<bos><start_of_turn>user{}<end_of_turn>
      <start_of_turn>model
      {}<end_of_turn><eos>""".format(content, example['response'][i]))
    return prompt_list

In [28]:
train_data = dataset['train']
print(generate_prompt(train_data[:1])[0])

<bos><start_of_turn>userBelow are sql tables schemas paired with instruction that describes a task. Using valid SQLite, write a response that appropriately completes the request for the provided tables.
Name the home team for carlton away team
CREATE TABLE table_name_77 (
    home_team VARCHAR,
    away_team VARCHAR
)<end_of_turn>
      <start_of_turn>model
      SELECT home_team FROM table_name_77 WHERE away_team = "carlton"<end_of_turn><eos>
